# 2. Tools calling

In this notebook, we will explore how the LLM can use external tools to enhance its capabilities. We'll begin by adding and configuring a search tool to allow the LLM to perform real-time searches.

By the end of this notebook, you'll be able to integrate and configure external search tools with a language model, and use them to perform real-time searches and enhance responses.

## User, LLM, Tools

![image](images/llm_tools_llama.jpeg)
Source: [Llama 3.1 Instruct](https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1)

1. **User Submits a Prompt**: The process starts when the user provides input or a prompt, which is sent to the **Executor** for handling.
2. **Executor Forwards Prompt to LLM**: The **Executor** forwards the user’s prompt to the **Llama Model** (LLM) for initial processing.
3. **LLM Processes the Prompt**: The **LLM** analyzes the prompt and determines whether it can generate a response directly or if external tools are needed to complete the request.
4. **LLM Requests External Tool Invocation**: If external data or additional resources are required (e.g., real-time data from a search engine or a code interpreter), the **Llama Model** signals the **Executor** to call the relevant tool.
5. **Executor Calls the Tools**: The **Executor** initiates the tool request, calling external tools (such as Brave Search, Wolfram Alpha, Code Interpreter, or other custom tools) to fetch necessary information.
6. **Tools Retrieve Data**: The external tools access the required information from outside environments (e.g., fetching real-time search results or performing complex calculations) and return the data to the **Executor**.
7. **Executor Synthesizes Final Response**: The **Executor** combines the original prompt, the **LLM**'s processing, and any external tool responses. The final response is then sent back to the **LLM**.
8. **LLM Generates Final Response**: The **LLM** synthesizes all the gathered data and generates a final response.
9. **Executor Sends Response to User**: The **Executor** delivers the final response back to the user, completing the interaction.

## 1. Setup

Before we begin, let's make sure your environment is set up correctly. We'll start by installing the necessary Python packages.

### Installing Required Packages

To get started, you'll need to install a few Python libraries. Run the following command to install them:

In [38]:
%pip install termcolor langchain_community

Note: you may need to restart the kernel to use updated packages.


These packages are used for:

- **termcolor:** Adding colored text output to the terminal, which is useful for debugging and improving the readability of logs and outputs.
- **langchain_community:** To enable the LLM to perform real-time searches, we need to install the `duckduckgo-search` library using the `langchain_community` package. This will allow the agent to search the web using DuckDuckGo.

## 2. Configuring a Simple Model

In this section, we configure the machine learning model that we will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16").

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables us to perform model-based tasks using the provided configuration.

In [39]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

### Invoking the Model

The `invoke_model` function handles the communication between the system and the language model. It takes two inputs: `sys_prompt` (system prompt) and `user_prompt` (the user's input), and performs the following steps:

1. **Prepare the Payload**: Combines the prompts into a format the model can understand.
2. **Send the Request**: Sends the prepared data to the model.
3. **Process the Response**: Once the model returns a response, it processes the output into a usable format.

The function returns the final processed response, which will be used to guide further actions.


In [40]:
def invoke_model(sys_prompt: str, user_prompt: str):
    """
    Prepare the payload, send the request to the model, and process the response.
    """
    # Prepare the payload
    payload = ollama_service.prepare_payload(
        user_prompt,
        sys_prompt,
    )

    # Invoke the model and get the response
    response_json = ollama_service.request_model_generate(
        payload,
    )

    # Process the model's response
    response_content = ollama_service.process_model_response(response_json)

    # Return the processed response
    return response_content

## 3. DuckDuckGo Search Tool

First, we will configure the tool by initializing the DuckDuckGo search functionality. This will be the primary tool for real-time search queries in this example.

In [41]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo Search Tool
duckduckgo_search = DuckDuckGoSearchRun()

# Verify tool name
print("Search Tool Name:", duckduckgo_search.name)

# Adding the search tool to the list of available tools
tools = [duckduckgo_search]

Search Tool Name: duckduckgo_search


In [42]:
# Render tool description to ensure it's ready for LLM usage
from langchain.tools.render import render_text_description_and_args

tools_name = duckduckgo_search.name

tools_description = (
    render_text_description_and_args(tools).replace("{", "{{").replace("}", "}}")
)
print("Tools Name:\n", tools_name)
print("Tools Description:\n", tools_description)

Tools Name:
 duckduckgo_search
Tools Description:
 duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query., args: {{'query': {{'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}}}


## 4. System Prompt for Tool Integration

Now that the DuckDuckGo search tool is set up, we need to write the system prompt to provide tool usage instructions. The LLM will reference this prompt when deciding whether to call the tool.


In [43]:
sys_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: {tools_name}
Cutting Knowledge Date: December 2023
Today's Date: 23 July 2024

---

You are an intelligent assistant designed to handle various tasks, including answering questions, providing summaries, and performing detailed analyses. All outputs must strictly be in JSON format.

---

## Tools
You have access to a variety of tools to assist in completing tasks. You are responsible for determining the appropriate sequence of tool usage to break down complex tasks into subtasks when necessary.

The available tools include:

{tools_description}

# Tool Instructions
- Always use the available tools when asked for real-time or updated information.
- If you choose to call a function ONLY reply in the following format:

{{
  "action": "Specify the tool you want to use.",
  "action_input": {{ # Provide valid JSON input for the action, ensuring it matches the tool’s expected format and data types.
    "key": "Value inputs to the tool in valid JSON format."
  }}
}}

Example:

{{
  "action": "duckduckgo_search",
  "action_input": {{
    "query": "Key features of large language models"
  }} 
}} 

Reminder:
- Do not include additional metadata such as `title`, `description`, or `type` in the `tool_input`
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line

You are a helpful assistant.<|eot_id|>
"""

### Format the System Prompt

Now we will insert the tool descriptions and finalize the system prompt by formatting it to include the correct tool name and description. This prompt will guide the LLM in calling the DuckDuckGo search tool when necessary.

In [44]:
formatted_sys_prompt = sys_prompt.format(
    tools_name=tools_name, tools_description=tools_description
)

## 5. LLM Request

We will now make a request, asking the LLM to search for the latest trends related to Large Language Models. This time, the LLM will have the ability to return the a payload that represents a call to the DuckDuckGo search tool to retrieve real-time information.

In [45]:
user_request = "What are the most recent Large Language Models?"

response = invoke_model(sys_prompt=sys_prompt, user_prompt=user_request)
print("LLM response:", response)


LLM response: {
    "action": "hugging_face_hub",
    "action_input": {
        "model": "list-models"
    }
}


## 6. Processing Tool Actions

After the LLM provides its response, we will process the action request. If the LLM calls the DuckDuckGo search tool, we will execute the function and retrieve the results. The tool's output will then be formatted and displayed.

In [46]:
import json

response_dict = json.loads(response)
action = response_dict["action"]
action_input = response_dict["action_input"]

for tool in tools:
    if tool.name == action:
        print(tool.name)
        try:
            result = tool.invoke(action_input)
            result_message = (
                f"""<|start_header_id|>ipython<|end_header_id|>\n\n{result}<|eot_id|>"""
            )
            print(result_message)
        except Exception as e:
            print(f"Error executing tool {action}: {str(e)}")
    else:
        print(f"Tool {action} not found or unsupported operation.")

Tool hugging_face_hub not found or unsupported operation.


## Conclusion
 
In this notebook, we've successfully extended the capabilities of the LLM by integrating an external tool (DuckDuckGo search). We covered:

- **LLM and Tool Integration:** How to modify the system prompt for tool access.
- **Tool Setup and Configuration:** Setting up the DuckDuckGo search tool for real-time information retrieval.
- **Making Requests and Handling Responses:** Using the tool during LLM interactions and processing the results.

With these tools in place, you're now equipped to extend the LLM's capabilities even further by adding more tools and refining its behavior. Happy coding!